In [5]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

def plot_features(img1, img2, fet1, fet2):
    img1_copy = np.copy(img1)  

  
    for corner in fet1:
        coord = (int(np.round(corner[0])), int(np.round(corner[1])))
        cv.circle(img=img1_copy, center=coord, radius=5, color=(255, 0, 0), thickness=5)

    img2_copy = np.copy(img2)  

  
    for corner in fet2:
        coord = (int(np.round(corner[0])), int(np.round(corner[1])))
        cv.circle(img=img2_copy, center=coord, radius=5, color=(255, 0, 0), thickness=5)

   
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.imshow(img1_copy, 'gray')
    plt.subplot(122)
    plt.imshow(img2_copy, 'gray')
    plt.show()

def get_keypoints(img):

    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img, None)
    keypoints_img1 = cv.drawKeypoints(img, kp1, None)
    plt.imshow(keypoints_img1)
    plt.show()
    return kp1, des1

def match_keypoints(kp1, kp2, des1, des2, img1, img2):
    
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
   
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    pts1 = []
    pts2 = []
    
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.8 * n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    return pts1, pts2

def draw_epilines(img1,img2,lines,pts1,pts2):
   
    img1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    img2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    r,c = img1.shape
    img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
    img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2